In [1]:
# used the human gut public data as basis here and the bin2cell version 0.3.3
import matplotlib.pyplot as plt
import pandas as pd 
import scanpy as sc
import numpy as np
import os
import bin2cell as b2c
import scanpy as sc
import cv2
print(b2c.__version__)

0.3.3


# Read the objects 

In [ ]:
path02 = ""
os.chdir(path=path02)
adata = sc.read_h5ad('2um_crc.h5ad') # read 2um object which has b2c cell grouping - see tutorial - https://nbviewer.org/github/Teichlab/bin2cell/blob/main/notebooks/demo.ipynb 
cdata = sc.read_h5ad('b2c_crc.h5ad') # read b2c object
anno_df = pd.read_csv('Graph-Based.csv',index_col=0)
anno = 'Graph-based'
cell_grouping = 'labels_joint'

# import annotations from loupe browser 

In [18]:
def map_loupe_to_b2c(
        adata,
        cdata,
        anno_df,
        anno,
        cell_grouping = 'labels_he_expanded', 
        ):
     

    # Calculate the binned row and column indices in adata
    adata.obs['binned_8_row'] = adata.obs['array_row'] // 4
    adata.obs['binned_8_col'] = adata.obs['array_col'] // 4

    # Create a combined index in adata and set it as the index
    adata.obs['mapping_index_08'] = 's_008um_' + adata.obs['binned_8_row'].astype(str).str.zfill(5) + '_' + adata.obs['binned_8_col'].astype(str).str.zfill(5) + '-1'
    adata.obs['mapping_index_08'].value_counts() # most should have 16 but there are some on the tissue edge that actually have less due to partial overlap 

    # read annotations from df and map to the 2um object 
    print('annotations to map:')
    print(anno_df.head(5))
    mapping_dict = pd.Series(anno_df[anno].values,index=anno_df.index).to_dict()
    adata.obs[anno] = adata.obs['mapping_index_08'].map(mapping_dict) # update 2um object 

    # assign annotation to cells by pandas.Series.mode(). The mode is the value that appears most often which captures the most frequent annotation for that cell from 2um data
    filtered_data = adata.obs[(adata.obs[cell_grouping] != 0) & adata.obs[anno].notna()]
    cell_annotation = (
        filtered_data.groupby(cell_grouping)[anno]
        .apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    )
    cell_annotation.index = cell_annotation.index.astype('str')
    cdata.obs[anno] = cdata.obs.index.map(cell_annotation).astype('category') # label cells 
    print('Annotations in b2c object:')
    print(cdata.obs[anno].value_counts())

map_loupe_to_b2c(adata,cdata,anno_df,anno,cell_grouping)


annotations to map:
                      Graph-based
Barcode                          
s_008um_00301_00321-1   Cluster 9
s_008um_00526_00291-1   Cluster 1
s_008um_00078_00444-1   Cluster 8
s_008um_00128_00278-1   Cluster 9
s_008um_00052_00559-1   Cluster 5
Annotations in b2c object:
Graph-based
Cluster 1     63924
Cluster 10    27159
Cluster 6     26112
Cluster 7     25441
Cluster 3     23434
Cluster 5     20737
Cluster 9     18089
Cluster 4     17776
Cluster 8     14151
Cluster 12     7268
Cluster 11     5024
Cluster 2      3926
Cluster 14     1986
Cluster 13     1801
Cluster 15      387
Name: count, dtype: int64


# Plot result

In [19]:
 
def save_outputs(image, legends, 
                 image_path="annotated_image.jpg", 
                 legend_paths=("fill_legend.jpg", "border_legend.jpg"), 
                 image_quality=75, 
                 image_compression=True):
    """
    Save the annotated image and legends to disk with optional compression.

    Parameters
    ----------
    image : numpy.ndarray
        The annotated image as a NumPy array.
    legends : dictionary of matplotlib.figure.Figure
        A list of legends as Matplotlib figures.
    image_path : str, optional
        Path to save the annotated image, by default "annotated_image.jpg".
    legend_paths : tuple of str, optional
        Paths to save the legends, by default ("fill_legend.jpg", "border_legend.jpg").
    image_quality : int, optional
        Quality of the saved image (1-100), by default 75.
    image_compression : bool, optional
        Whether to enable compression for the image, by default True.
    """
    from PIL import Image
    import logging 
    logging.getLogger().setLevel(logging.WARNING)

    # Save the annotated image
    img_pil = Image.fromarray(image)
    img_pil.save(
        image_path,
        format="JPEG",
        quality=image_quality,
        optimize=image_compression
    )
    print(f"Image saved to {image_path} with quality={image_quality}, compression={image_compression}")

    # Save each legend
    for legend, legend_path in zip(legends.values(), legend_paths):
        # Ensure the file extension is PNG for better Matplotlib support
        if not legend_path.endswith(".pdf"):
            legend_path = legend_path.rsplit('.', 1)[0] + ".pdf"
        
        legend.savefig(legend_path, dpi=300, bbox_inches="tight")
        print(f"Legend saved to {legend_path}")

In [20]:
import matplotlib
image_path = path02+"/stardist/he.tiff"
labels_npz_path = path02+"/stardist/he.npz"
matplotlib.rcParams['pdf.fonttype'] = 42
sc.settings.set_figure_params(dpi = 150, color_map = 'RdPu', dpi_save = 150, vector_friendly = True, format = 'pdf')

In [21]:
import time
t1=time.time()
# make annotated image of cell labels and confidance scores
fill_col=anno
annotated_image, legends = b2c.view_cell_labels(image_path, labels_npz_path, cdata, fill_key=fill_col,cat_cmap='tab20') 
t2=time.time()
t2-t1

10.376625061035156

In [22]:
# Save outputs
save_outputs(
    image=annotated_image,
    legends=legends,
    image_path=f"annotated_image_{anno}.jpg",
    legend_paths=("fill_legend.pdf", "border_legend.pdf"),
    image_quality=90,
    image_compression=True
)

Image saved to annotated_image_Graph-based.jpg with quality=90, compression=True
Legend saved to fill_legend.pdf
